In [ ]:
import json
import os
import logging
import pandas as pd
from dotenv import load_dotenv
from typing import Dict
from pprint import pprint
from pathlib import Path
from llm_assessor import *
from langchain.chat_models import ChatOpenAI

load_dotenv(override=True)

In [ ]:
# Set working directory - Not required if using Jupyter outside of VScode
workdir = os.environ["workdir"]
os.chdir(workdir)
os.getcwd()

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(process)d-%(levelname)s-%(message)s',
    datefmt="%Y-%m-%d %H:%M:%S",
    )


# Restructure AQA History Mark Scheme

In [ ]:
hist_mark_scheme = ["""
Level 2: Developed analysis of source based on content and/or provenance 3–4
Students may progress from a simple analysis of the source with extended
reasoning supported by factual knowledge and understanding related to the
features of the source.
For example, the cartoon is critical of the League of Nations because it was
produced at the time of the Manchurian Crisis when the League failed to stop
the Japanese invasion of Manchuria. The title of the cartoon suggests that Japan
is ignoring the League of Nations.
Level 1: Simple analysis of source based on content and/or provenance 1–2
Students identify relevant features in the source and support them with simple
factual knowledge and understanding.
For example, the cartoon shows that Japan is walking all over the League of
Nations.
Students either submit no evidence or fail to address the question 0
""",
"""
Level 2: Developed analysis of source based on content and/or provenance
Students may progress from a simple analysis of the source with extended
reasoning supported by factual knowledge and understanding related to the
features of the source.
For example, the cartoon is British and the title refers to being betrayed by
Russia who was one of their allies. The Bolshevik Revolution occurred in 1917
and Lenin made peace with Germany in 1918. This reduced the demands on
the German army in the East and allowed them to concentrate all their forces
against the Allies on the Western Front.
3–4
Level 1: Simple analysis of source based on content and/or provenance
Students identify relevant features in the source and support them with simple
factual knowledge and understanding.
For example, the Bolshevik is dragging Russia towards Germany against her
will; the cartoon was published in the year Russia withdrew from the First
World War.
1–2
Students either submit no evidence or fail to address the question 0
""",
"""
Target Analyse sources contemporary to the period (AO3a)
Evaluate sources and make substantiated judgements (AO3b)
In analysing and evaluating sources, students will draw on their contextual
knowledge to question critically the content and provenance of sources (for
example, the context of the time in which source was created, place, author’s
situation, knowledge, beliefs, circumstances, access to information, purpose
and audience).
Level 4: Complex evaluation of both sources with sustained judgement 10–12
based on content and provenance
Students may progress from a developed evaluation of the sources by complex
reasoning related to utility on the basis of content and provenance. They may
evaluate the relationship between the sources based on analysis of provenance
and contextual knowledge.
For example, the sources are useful to show how difficult it was for the
governments of the time to respond to the threats posed by Hitler. The Munich
Agreement mentioned in Source B was the work of several countries, not just
Britain alone, therefore the responsibility for giving in to his demands is a
shared one. The American view in Source C is useful to show that even countries
outside Europe were concerned with the issue of Czechoslovakia and the policy
of appeasement.
Level 3: Developed evaluation of sources based on the content and/or 7-9
provenance
Students may progress from a simple evaluation of the sources with extended
reasoning related to utility on the basis of content and/or provenance.
For example, Source B is useful for showing a positive view of appeasement as
Chamberlain presented it to the people of Britain. The Munich Agreement
allowed Hitler to occupy the Sudetenland in Czechoslovakia despite it not being
German land. However, in return for this concession, Chamberlain secured the
Anglo-German declaration which was a promise that war would be avoided.
Source C is critical of Chamberlain for thinking that Hitler will be content with
the concessions. The Munich Agreement had allowed Hitler to take over land in
Czechoslovakia. Hitler had promised not to make any further demands for more
territory but many people at the time thought he would try and take more land.
For example, Source C is useful because it shows an opinion from outside
Europe that was critical of Chamberlain’s policy of appeasement. The cartoon
reflects the view that by giving into Hitler the Munich Agreement would
encourage his territorial ambitions even further. This prediction came true
when Hitler invaded the rest of Czechoslovakia in 1939. Source B shows why, in
the short term, appeasement seemed to have worked. Chamberlain wanted to
use discussion to avoid war with Hitler and Source B was seen as evidence that
he had achieved his aim.
Level 2: Simple evaluation of source(s) based on content and/or provenance 4–6
Students may progress from a basic analysis of the source(s) to simple
evaluation of the content and/or provenance.
For example, Source B shows that appeasement seemed to have worked.
Chamberlain wanted to use discussion to avoid war with Hitler and this joint
statement was seen as evidence that he had achieved his aim.
For example, Source C is criticising appeasement. The American cartoon is
saying the Munich Agreement might not keep Hitler quiet for long. The Munich
Agreement had allowed Hitler to take over land in Czechoslovakia. Hitler had
promised not to make any further demands for more territory
Level 1: Basic analysis of sources(s) 1–3
Answers may show understanding/support for one or both sources, but the
case is made by assertion/basic inference.
Students identify basic features which are valid about the sources and related to
the enquiry point, for example, Source B says that Germany and Britain agreed
never to go to war.
Source C shows that Chamberlain is trying to keep Hitler quiet.
Students either submit no evidence or fail to address the question 0
""",
"""
Level 4: Complex analysis of causation/consequence 7–8
Answer is presented in a coherent narrative/account that demonstrates a
range of accurate and detailed knowledge and understanding that is
relevant to the question
Extends Level 3.
Students may progress from a developed narrative of causation/consequence
with complex sequencing and reasoning supported by a range of accurate and
detailed factual knowledge and understanding which might be related, for
example, to an analysis of how/why tension changed at different stages and/or
showing understanding about how much each part of the sequence reduced
tension.
For example, the contribution to international peace by the League sometimes
depended on the status of the country they were defending. In the issue of the
Corfu invasion by Mussolini, the League originally upheld the complaint by
Greece but this was undermined by the influence of more powerful countries
such as Britain and France who chose to support Italy’s interests.
Level 3: Developed analysis of causation/consequence 5–6
Answer is presented in a structured and well-ordered narrative/account
that demonstrates a range of accurate knowledge and understanding that
is relevant to the question.
Extends Level 2.
Students may progress from a simple narrative of causation/consequence with
developed sequencing and reasoning supported by a range of accurate factual
knowledge and understanding which might be related, for example to an
analysis of how/why tension was reduced at one stage in the process.
For example, the League contributed to peace because it helped to avoid a war
breaking out between Sweden and Finland. Both countries wanted to claim
ownership of the Aaland Islands. The League found a compromise that both
countries accepted.
Level 2: Simple analysis of causation/consequence 3–4
Answer is presented in a structured account that demonstrates specific
knowledge and understanding that is relevant to the question
Students may progress from a basic narrative of causation/consequence by
showing a simple understanding of sequencing, supporting it with factual
knowledge and understanding.
For example, the League of Nations created an international court of justice to
settle disagreements between countries. Every member of the Assembly could
vote to choose the judges.
Level 1: Basic analysis of causation/consequence 1–2
Answer is presented as general statements which demonstrates basic
knowledge and understanding that is relevant to the question
Students identify cause(s)/consequence(s) about the events such as the League
tried to prevent another war by discussing problems. The Commission for
Refugees helped prisoners of war to return to their homes.
Students either submit no evidence or fail to address the question 0
""",
"""
Level 4: Complex explanation of stated factor and other factor(s) leading to 13–16
a sustained judgement
Answer demonstrates a range of accurate and detailed knowledge and
understanding that is relevant to the question
Answer demonstrates a complex, sustained line of reasoning which has a
sharply-focused coherence and logical structure that is fully substantiated, with
well-judged relevance.
Extends Level 3.
Students may progress from a developed explanation of causation by complex
explanation of the relationship between causes supported by detailed factual
knowledge and understanding to form a sustained judgement.
For example, the loss of territory was not the main reason because Germany
expected some losses. What they hated the most about the Treaty of Versailles
was that Germany was excluded from the negotiations and it was a ‘Diktat’. This
meant they could not influence the severity of the resulting terms. This meant
they could not prevent the inclusion of Clause 231 concerning war guilt which
said they were responsible for starting the war.
Level 3: Developed explanation of the stated factor and other factor(s) 9-12
Answer demonstrates a range of accurate knowledge and understanding
that is relevant to the question
Answer demonstrates a developed, sustained line of reasoning which has
coherence and logical structure; it is well substantiated, and with sustained,
explicit relevance.
Extends Level 2.
Answer may suggest that one reason has greater merit.
Students may progress from a simple explanation of causation with developed
reasoning supported by factual knowledge and understanding.
For example, the territorial terms of the Treaty of Versailles were the main
reason why Germany hated it because they were humiliating and economically
damaging. 10% of its land was given to neighbouring countries. A strip of land
was given to Poland which effectively split Germany in two by creating a corridor
that created access to the sea. As well as losing land outright, Germany also lost
control of areas such as the coal rich Saarland which was put under a League of
Nations mandate for 15 years and the Rhineland was demilitarised.
For example, the reparations payments were another reason why Germans
hated the Treaty of Versailles because it would make it harder for Germany to
recover. People were suffering from starvation after the war, and they were
angry that the reparation bill was so huge. Germany had to pay money to the
Allies for the damage caused during the war. It was estimated that it would take
over 60 years to pay back the full bill.
Level 2: Simple explanation of stated factor or other factor(s) 5-8
Answer demonstrates specific knowledge and understanding that is
relevant to the question
Answer demonstrates a simple, sustained line of reasoning which is coherent,
structured, substantiated and explicitly relevant.
Answers arguing a preference for one judgement but with only basic
explanation of another view will be marked at this level.
Students may progress from a basic explanation of causation by simple
reasoning and supporting it with factual knowledge and understanding.
For example, the War Guilt clause was hated by the German people because it
forced them to accept the full blame for starting the war. Germany had no
choice about signing the treaty; they were threatened with renewed fighting if
they did not accept all the terms.
Level 1: Basic explanation of one or more factors 1–4
Answer demonstrates basic knowledge and understanding that is relevant
to the question
Answer demonstrates a basic line of reasoning, which is coherent, structured
with some substantiation; the relevance might be implicit.
Students recognise and provide a basic explanation of one or more factors.
Students may offer a basic explanation of the stated factor, such as the Treaty
of Versailles took a lot of land from Germany.
Students may offer basic explanations of other factor(s), for example, Germany
was humiliated and forced to reduce the size of its army.
Students either submit no evidence or fail to address the question 0
""",
"""
Spelling, punctuation and grammar
Performance descriptor Marks
awarded
High
performance
• Learners spell and punctuate with consistent accuracy
• Learners use rules of grammar with effective control
 of meaning overall
• Learners use a wide range of specialist terms as appropriate
4 marks
Intermediate
performance
• Learners spell and punctuate with considerable accuracy
• Learners use rules of grammar with general control
 of meaning overall
• Learners use a good range of specialist terms as
 appropriate
2–3
marks
Threshold
performance
• Learners spell and punctuate with reasonable accuracy
• Learners use rules of grammar with some control of
 meaning and any errors do not significantly hinder meaning
 overall
• Learners use a limited range of specialist terms as
 appropriate
1 mark
No marks
awarded
• The learner writes nothing
• The learner’s response does not relate to the question
• The learner’s achievement in SPaG does not reach the
 threshold performance level, for example errors in spelling,
 punctuation and grammar severely hinder meaning
0 marks
"""
]

In [ ]:
aqa_hist_ms = {
    "explain" : 
    {
        "option_c": 
        [
        """
        Target Analyse sources contemporary to the period (AO3a)
        Level 2: Developed analysis of source based on content and/or provenance 3–4
        Students may progress from a simple analysis of the source with extended
        reasoning supported by factual knowledge and understanding related to the
        features of the source.
        For example, the cartoon is critical of Stalin because it was published during the
        Korean War when the USA and USSR were fighting over the spread of
        communism in Asia. Stalin provided weapons for Chinese soldiers to use in the
        fight against the UN forces in South Korea.
        Level 1: Simple analysis of source based on content and/or provenance 1–2
        Students identify relevant features in the source and support them with simple
        factual knowledge and understanding.
        For example, the source shows that Stalin is helping Mao to send soldiers to
        fight and die. The cartoon is American so it will show communist countries in a
        bad light.
        Students either submit no evidence or fail to address the question 0  
        """.strip(),
        """
        Level 4: Complex evaluation of both sources with sustained judgement 10–12
        based on content and provenance
        Students may progress from a developed evaluation of the sources by complex
        reasoning related to utility on the basis of content and provenance. They may
        evaluate the relationship between the sources based on analysis of provenance
        and contextual knowledge.
        For example, taken together the sources are useful because they show how the
        Berlin Crisis was seen differently by both sides in the Cold War. An historian can
        see how even though the Blockade did not lead to armed conflict, Sources B and
        C are evidence that both sides tried to blame each other for the incident. The
        sources both show how tension increased between East and West during the
        Berlin Blockade and made future cooperation less likely.
        Level 3: Developed evaluation of sources based on the content and/or 7–9
        provenance
        Students may progress from a simple evaluation of the sources with extended
        reasoning related to utility on the basis of content and/or provenance.
        For example, Source B is useful to show how Stalin tried to justify his actions by
        accusing the Western powers of acting unreasonably. He is trying to blame the
        Western powers for breaking the agreement made at the Yalta Conference in
        1945, that Germany and Berlin would be kept as four separate sections.
        However, the Western Allies merged their zones in 1948 and also introduced a
        single currency into the newly formed West Germany and West Berlin. Source C
        is a British cartoon so it will be opposed to Stalin’s blockade of West Berlin. The
        cartoon makes fun of ‘Joe’ Stalin because he cannot block the air corridors to
        stop planes from bringing supplies to Berlin.
        For example, Source C is useful to an historian because it shows that British
        people were pleased to be standing up to Stalin’s Blockade. The Allies began the
        Berlin Airlift which was a solution to the blockade of land routes to West Berlin.
        Planes were used to provide essential supplies of food and fuel to the people of
        West Berlin. The cartoon shows that Stalin wanted to try and stop the planes
        from flying. However, he did not want to risk shooting them down as that would
        be seen as an act of war. Source B explains that Stalin’s motive for the blockade
        was to protect his zone of occupation. When Britain, France and the USA
        merged their zones into one, Stalin was afraid that Germany would recover and
        could threaten the USSR once again.
        Level 2: Simple evaluation of source(s) based on content and/or provenance 4–6
        Students may progress from a basic analysis of the source(s) to simple
        evaluation of the content and/or provenance.
        For example, Source B explains that Stalin’s motive for the blockade was to
        protect his zone. When Britain, France and the USA merged their zones into
        one, Stalin was afraid that Germany would recover and could threaten the USSR
        once again.
        Source C is a British cartoon so it will be opposed to Stalin’s blockade of West
        Berlin. The cartoon makes fun of ‘Joe’ Stalin because he cannot block the air
        corridors to stop planes from bringing supplies to Berlin.
        Level 1: Basic analysis of sources(s) 1–3
        Answers may show understanding/support for one or both sources, but the
        case is made by assertion/basic inference.
        Students identify basic features which are valid about the sources and related to
        the enquiry point, for example, Source B says that the Soviet Government is
        trying to look after the people of Berlin.
        Source C shows that Stalin is responsible for blocking access to Berlin.
        Students either submit no evidence or fail to address the question 0
        """.strip(),
        ],
        "option_d": 
        [
        """
        Level 2: Developed analysis of source based on content and/or provenance 3–4
        Students may progress from a simple analysis of the source with extended
        reasoning supported by factual knowledge and understanding related to the
        features of the source.
        For example, the cartoon is criticising America for the events that took place at
        Kent State University in 1970. The soldier is making an excuse for his use of
        force. The figure of Uncle Sam represents the people of America; he looks
        ashamed because students who were making an anti-war demonstration were
        shot at by National Guard. Several students were killed.
        Level 1: Simple analysis of source based on content and/or provenance 1–2
        Students identify relevant features in the source and support them with simple
        factual knowledge and understanding.
        For example, the cartoon shows an American soldier who has just used his gun
        against people who did not have proper weapons. The guards shot students
        who were protesting.
        Students either submit no evidence or fail to address the question 0
        """.strip(),
        """
        Level 4: Complex evaluation of both sources with sustained judgement based on
        content and provenance 10–12
        Students may progress from a developed evaluation of the sources by complex
        reasoning related to utility on the basis of content and provenance. They may
        evaluate the relationship between the sources based on analysis of provenance
        and contextual knowledge.
        For example, the sources illustrate how the civil war in Korea became a proxy
        war between the USSR and the USA as part of the Cold War. In Source B, Stalin
        is only offering to provide equipment rather than troops because he did not
        want direct confrontation with the USA. That is also the reason for the message
        being confidential. The context of Source C is that President Truman had issued
        a doctrine to say that America would follow a policy to ‘contain’ the spread of
        communism. However, the propaganda poster shows that the USA did not want
        to be seen as starting a war against communism because it shows North Korea
        as the aggressor. It stresses that the action against North Korea has worldwide
        support and the USA was acting with the authorisation of the UN.
        Level 3: Developed evaluation of sources based on the content and/or 7-9
        provenance
        Students may progress from a simple evaluation of the sources with extended
        reasoning related to utility on the basis of content and/or provenance.
        For example, Source B is useful to show an historian why North Korea was so
        successful in the first stages of the war. With support from the USSR, North
        Korea had built up its own army and trained troops whereas South Korea was
        not as strong. The North Korean army advanced rapidly in June-September 1950
        and captured most of South Korea. Source C was published in America to show
        the actions of the UN in a positive light. America was in charge of the UN forces
        that landed in South Korea to help drive the North Koreans back to the border.
        For example, Source C is a form of propaganda that is justifying the need to
        fight against communism. In 1950, the UN had voted to take military action
        against North Korea. The USSR refused to attend meetings at this time and so
        they had not been able to veto this decision. Source B shows the Korean War
        started because the USSR provided military equipment that allowed the North
        Korean Army to invade South Korea. North Korea was communist and Stalin
        was happy to support an attempt to unite both halves of Korea under
        communist leadership.
        Level 2: Simple evaluation of source(s) based on content and/or provenance 4-6
        Students may progress from a basic analysis of the source(s) to simple
        evaluation of the content and/or provenance.
        For example, Source B shows why the Korean War started in 1950 because the
        USSR provided military equipment that allowed the North Korean Army to
        invade capitalist South Korea. North Korea was communist and Stalin was
        happy to support an attempt to unite both halves of Korea under communist
        leadership.
        Source C was published in America to show the actions of the UN in a positive
        light. America was in charge of the UN forces that landed in South Korea to help
        drive the North Koreans back to the border.
        Level 1: Basic analysis of sources(s) 1-3
        Answers may show understanding/support for one or both sources, but the
        case is made by assertion/basic inference.
        Students identify basic features which are valid about the sources and related to
        the enquiry point, for example, Source B says that Stalin was willing to help Kim
        Il Sung with the issue of South Korea.
        Source C shows that the UN is trying to stop the Communists from attacking
        Korea.
        Students either submit no evidence or fail to address the question 0
        """.strip()
        ],
        "option_e": 
        [
        """
        Level 2: Developed analysis of source based on content and/or provenance 3–4
        Students may progress from a simple analysis of the source with extended
        reasoning supported by factual knowledge and understanding related to the
        features of the source.
        For example, the cartoon was produced when a new President had taken over
        and so will criticise the previous one for his mistakes. It suggests that Bush’s
        memoirs might avoid issues about the real problems facing Iraq. Iraq was
        suffering from many civilian deaths due to the continuing Insurgency and a
        refugee crisis.
        Level 1: Simple analysis of source based on content and/or provenance 1–2
        Students identify relevant features in the source and support them with simple
        factual knowledge and understanding.
        For example, the cartoon shows that Obama’s view that Bush had left Iraq in a
        ‘mess’ and the image makes fun of him holding a crayon and not remembering
        things as they really were.
        Students either submit no evidence or fail to address the question 0
        """.strip(),
        """
        Level 4: Complex evaluation of both sources with sustained judgement 10–12
        based on content and provenance
        Students may progress from a developed evaluation of the sources by complex
        reasoning related to utility on the basis of content and provenance. They may
        evaluate the relationship between the sources based on analysis of provenance
        and contextual knowledge.
        For example, taken together the sources are useful because they illustrate how
        the US and its allies quickly reacted to the attacks. Source B shows the
        confidence that Americans had in their power in the face of a terrified enemy.
        The artist that created Source B was trying to paint an image of the ability of the
        US to assert its strength. This contrasts with Source C which looked for a wider
        response and urged countries to stand together and work with the UN to
        undermine and weaken terrorism. However, the reality was that while NATO
        stood with the US, the UN was more cautious, despite Blair’s appeal, it did not
        vote for military action.
        Level 3: Developed evaluation of sources based on the content and/or 7-9
        provenance
        Students may progress from a simple evaluation of the sources with extended
        reasoning related to utility on the basis of content and/or provenance.
        For example, the cartoon is useful for showing the opinion that the US had
        tremendous power which it would use to defeat terrorism. The purpose of the
        cartoon was to appeal to American patriotism and encourage support for
        whatever measures would be taken to deal with the threat. The suggestion is
        that the way terrorism is caricatured shows that America would be victorious
        over a weak and frightened enemy.
        Source C is useful for giving information about the wider reaction to the attacks.
        For example, Blair considered that Britain was already at war because of the
        attack on British civilians in the Twin Towers. NATO members also joined the
        chorus of anger and sent forces as part of the coalition to Afghanistan.
        Level 2: Simple evaluation of source(s) based on content and/or provenance 4–6
        Students may progress from a basic analysis of the source(s) to simple
        evaluation of the content and/or provenance.
        For example, Source B was published in America. It gives an American view that
        in 2001 the US was ready to use the great power it had to pursue and defeat
        terrorists.
        For example, Source C shows that Blair, as an ally of the US and close friend of
        Bush, was calling on countries to form a coalition and work together to defeat
        terrorists. For example, NATO forces came together to support the US-UK
        operations in Afghanistan.
        Level 1: Basic analysis of sources(s) 1–3
        Answers may show understanding/support for one or both sources, but the
        case is made by assertion/basic inference.
        Students identify basic features which are valid about the sources and related to
        the enquiry point, for example, Source B says that the US would now use its
        great power against terrorism.
        Source C shows that countries should work together to fight terrorism.
        Students either submit no evidence or fail to address the question 0
        """.strip()
        ]
    },
    "analyse": 
    {
        "option_c": 
        [
            """
            Level 4: Complex analysis of causation/consequence 7–8
            Answer is presented in a coherent narrative/account that demonstrates a
            range of accurate and detailed knowledge and understanding that is
            relevant to the question
            Extends Level 3.
            Students may progress from a developed narrative of causation/consequence
            with complex sequencing and reasoning supported by a range of accurate and
            detailed factual knowledge and understanding which might be related, for
            example, to an analysis of how/why tension increased at different stages and
            /or showing understanding about how much each part of the sequence
            increased tension.
            For example, the Cold War was negatively affected because the events of 1956
            demonstrated that the United Nations was unable to prevent acts of brutality by
            the USSR. The UN passed a motion to call for the withdrawal of Soviet tanks
            from Hungary but the USSR vetoed the decision.
            Level 3: Developed analysis of causation/consequence 5–6
            Answer is presented in a structured and well-ordered narrative/account
            that demonstrates a range of accurate knowledge and understanding that
            is relevant to the question
            Extends Level 2.
            Students may progress from a simple narrative of causation/consequence with
            developed sequencing and reasoning supported by a range of accurate factual
            knowledge and understanding which might be related, for example to an
            analysis of how/why tension increased at one stage in the process.
            For example, the Cold War was affected because the USA did not intervene to
            help the Hungarian revolutionaries in 1956. The USA argued that since Hungary
            was in the Soviet ‘sphere of influence’, it would be politically unwise if America
            was to interfere.
            Level 2: Simple analysis of causation/consequence 3–4
            Answer is presented in a structured account that demonstrates specific
            knowledge and understanding that is relevant to the question
            Students may progress from a basic narrative of causation/consequence by
            showing a simple understanding of sequencing, supporting it with factual
            knowledge and understanding.
            For example, Nagy became the leader of Hungary. He made democratic style
            reforms that gave people more political freedom and he announced that
            Hungary would leave the Warsaw Pact.
            Level 1: Basic analysis of causation/consequence 1–2
            Answer is presented as general statements which demonstrates basic
            knowledge and understanding that is relevant to the question
            Students identify cause(s)/consequence(s) about the events such as Soviet tanks
            and troops were sent to Hungary to crush the uprising.
            Students either submit no evidence or fail to address the question 0
            """.strip()
        ],
        "option_d":
        [
            """
            Level 4: Complex analysis of causation/consequence 7–8
            Answer is presented in a coherent narrative/account that demonstrates a
            range of accurate and detailed knowledge and understanding that is
            relevant to the question
            Extends Level 3.
            Students may progress from a developed narrative of causation/consequence
            with complex sequencing and reasoning supported by a range of accurate and
            detailed factual knowledge and understanding which might be related, for
            example, to an analysis of how/why tension increased at different stages and
            /or showing understanding about how much each part of the sequence
            increased tension.
            For example, President Ford was still able to exert a negative influence after the
            war ended in 1973 because he opposed Vietnam joining the United Nations. It
            was a problem for Vietnam because it remained isolated from the international
            community and made recovery harder.
            Level 3: Developed analysis of causation/consequence 5–6
            Answer is presented in a structured and well-ordered narrative/account
            that demonstrates a range of accurate knowledge and understanding that
            is relevant to the question
            Extends Level 2.
            Students may progress from a simple narrative of causation/consequence with
            developed sequencing and reasoning supported by a range of accurate factual
            knowledge and understanding which might be related, for example to an
            analysis of how/why tension increased at one stage in the process.
            For example, one problem was that the soil remained contaminated after the
            war because the farmland and jungle in Vietnam had been destroyed using
            chemical agents. The chemicals caused cancers and other illnesses in successive
            generations of Vietnamese people.
            Level 2: Simple analysis of causation/consequence 3–4
            Answer is presented in a structured account that demonstrates specific
            knowledge and understanding that is relevant to the question
            Students may progress from a basic narrative of causation/consequence by
            showing a simple understanding of sequencing, supporting it with factual
            knowledge and understanding.
            For example, thousands of children were born as a result of relationships
            between Vietnamese women and US soldiers. Many faced prejudice for the rest
            of their lives because they were a living reminder of the conflict.
            Level 1: Basic analysis of causation/consequence 1–2
            Answer is presented as general statements which demonstrates basic
            knowledge and understanding that is relevant to the question
            Students identify cause(s)/consequence(s) about the events such as Unexploded
            bombs remained in the soil and they could be detonated when the land was
            farmed.
            Students either submit no evidence or fail to address the question 0
            """.strip()
        ],
        "option_e":
        [
            """
            Level 4: Complex analysis of causation/consequence 7–8
            Answer is presented in a coherent narrative/account that demonstrates a
            range of accurate and detailed knowledge and understanding that is
            relevant to the question
            Extends Level 3.
            Students may progress from a developed narrative of causation/consequence
            with complex sequencing and reasoning supported by a range of accurate and
            detailed factual knowledge and understanding which might be related, for
            example, to an analysis of how/why tension increased at different stages and
            /or showing understanding about how much each part of the sequence
            increased tension and led to a crisis.
            For example, problems were caused for the western coalition as it felt it had to
            restrain Saddam by imposing no fly zones over Iraq. However, this led to further
            problems as neighbouring Arab states became increasingly concerned about
            western actions and some adopted a more hostile attitude to the US and Britain
            shown, for example, by attacks on embassies.
            Level 3: Developed analysis of causation/consequence 5–6
            Answer is presented in a structured and well-ordered narrative/account
            that demonstrates a range of accurate knowledge and understanding that
            is relevant to the question
            Extends Level 2.
            Students may progress from a simple narrative of causation/consequence with
            developed sequencing and reasoning supported by a range of accurate factual
            knowledge and understanding which might be related, for example to an
            analysis of how/why tension increased at one stage in the process.
            For example, problems were caused for the international community which felt
            it had to intervene and restrain Saddam who was responsible for sectarian
            violence. No fly zones were imposed by the US, Britain and France to stop
            Saddam’s attacks.
            Level 2: Simple analysis of causation/consequence 3–4
            Answer is presented in a structured account that demonstrates specific
            knowledge and understanding that is relevant to the question
            Students may progress from a basic narrative of causation/consequence by
            showing a simple understanding of sequencing, supporting it with factual
            knowledge and understanding.
            For example, in order to maintain Sunni control, biological weapons were used
            by Saddam Hussein against minority groups, forcing many to flee the country.
            Level 1: Basic analysis of causation/consequence 1–2
            Answer is presented as general statements which demonstrates basic
            knowledge and understanding that is relevant to the question
            Students identify cause(s)/consequence(s) about the events such as both groups
            claimed that they were treated as second class citizens and had no civil rights.
            Students either submit no evidence or fail to address the question 0
            """.strip()
        ]
    },
    "judgement": 
    {
        "option_c": 
        [
            """
            Level 4: Complex explanation of stated factor and other factor(s) leading to 13–16
            a sustained judgement
            Answer demonstrates a range of accurate and detailed knowledge and
            understanding that is relevant to the question
            Answer demonstrates a complex, sustained line of reasoning which has a
            sharply-focused coherence and logical structure that is fully substantiated, with
            well-judged relevance.
            Extends Level 3.
            Students may progress from a developed explanation of causation by complex
            explanation of the relationship between causes supported by detailed factual
            knowledge and understanding to form a sustained judgement.
            For example, the U2 Crisis was not the main reason why tension developed
            during the 1960s because more serious conflicts followed such as the loss of life
            incurred during the Prague Spring and the international fear of nuclear war
            during the Cuban Missile Crisis. However, coming as it did at the very start of
            the decade, it could be argued that it set the tone for East–West relations by
            making it clear that ‘peaceful co–existence’ was unlikely.
            Level 3: Developed explanation of the stated factor and other factor(s) 9–12
            Answer demonstrates a range of accurate knowledge and understanding
            that is relevant to the question
            Answer demonstrates a developed, sustained line of reasoning which has
            coherence and logical structure; it is well substantiated, and with sustained,
            explicit relevance.
            Extends Level 2.
            Answer may suggest that one reason has greater merit.
            Students may progress from a simple explanation of causation with developed
            reasoning supported by factual knowledge and understanding.
            For example, the U2 Crisis was the main reason why tension developed
            between the East and West because once the spy plane had been captured by
            the Soviets, the American government lied about the purpose of the flight. They
            claimed it was simply conducting weather research. Khrushchev revealed the
            deception at the opening of the Paris Peace summit and when Eisenhower
            refused to apologise, the meeting was abandoned. This affected the Cold War
            because it was a missed opportunity to have face-to-face discussions and
            reduce tension.
            For example, the Prague Spring was also a reason why tension developed
            during the 1960s because Soviet forces invaded Czechoslovakia and installed
            their own loyal government. Dubcek had introduced political reforms into
            Czechoslovakia by relaxing state control of the economy and the press. The
            USSR and the other Warsaw Pact leaders felt any reforms could damage the
            communist system as a whole. East–West tension increased when Western
            countries criticised the violent response from Moscow and called for the UN to
            condemn the USSR.
            Level 2: Simple explanation of stated factor or other factor(s) 5-8
            Answer demonstrates specific knowledge and understanding that is
            relevant to the question
            Answer demonstrates a simple, sustained line of reasoning which is coherent,
            structured, substantiated and explicitly relevant.
            Answers arguing a preference for one judgement but with only basic
            explanation of another view will be marked at this level.
            Students may progress from a basic explanation of causation by simple
            reasoning and supporting it with factual knowledge and understanding.
            For example, tension developed during the 1960s because of the Cuban Missile
            Crisis. The USSR had nuclear missiles on the island of Cuba which put America in
            danger of attack. Kennedy used a naval blockade to stop any more missiles
            arriving on Cuba and demanded that the existing missiles were removed.
            Level 1: Basic explanation of one or more factors 1–4
            Answer demonstrates basic knowledge and understanding that is relevant
            to the question
            Answer demonstrates a basic line of reasoning, which is coherent, structured
            with some substantiation; the relevance might be implicit.
            Students recognise and provide a basic explanation of one or more factors.
            Students may offer a basic explanation of the stated factor, such as tension
            developed between East and West during the 1960s because an American U2
            plane was caught spying on the USSR.
            Students may offer basic explanations of other factor(s), for example, there was
            an increase in tension when USSR built the Berlin Wall. No one knew how the
            USA would react. Fighting may have broken out.
            Students either submit no evidence or fail to address the question 0
            """.strip()
        ],
        "option_d":
        [
            """
            Level 4: Complex explanation of stated factor and other factor(s) leading 13–16
            to a sustained judgement
            Answer demonstrates a range of accurate and detailed knowledge and
            understanding that is relevant to the question .
            Answer demonstrates a complex, sustained line of reasoning which has a
            sharply focused coherence and logical structure that is fully substantiated, with
            well-judged relevance.
            Extends Level 3.
            Students may progress from a developed explanation of causation by complex
            explanation of the relationship between causes supported by detailed factual
            knowledge and understanding to form a sustained judgement.
            For example, the Gulf of Tonkin incident was just one of the reasons for
            escalation in the 1960s. The wider context was that as the war continued, the
            nature of the conflict made decisive victory impossible. The size and strength of
            US military power did not bring a great advantage in jungle warfare against the
            guerrilla tactics of the Vietcong, nor could it destroy the Ho Chi Minh trail.
            Level 3: Developed explanation of the stated factor and other factor(s) 9–12
            Answer demonstrates a range of accurate knowledge and understanding
            that is relevant to the question
            Answer demonstrates a developed, sustained line of reasoning which has
            coherence and logical structure; it is well substantiated, and with sustained,
            explicit relevance.
            Extends Level 2.
            Answer may suggest that one reason has greater merit.
            Students may progress from a simple explanation of causation with developed
            reasoning supported by factual knowledge and understanding.
            For example, the Gulf of Tonkin incident involved a torpedo attack on USS
            Maddox and the report of another imminent attack. Although no further attacks
            occurred, the suspicion was sufficient for the US government to pass a
            resolution which gave the President the power to take further military against
            North Vietnam. This escalated the conflict because President Johnson used this
            power to launch bombing campaigns against North Vietnam and sent
            thousands of American soldiers to South Vietnam.
            For example, the main reason the conflict continued in the 1960s was because
            the Vietcong successfully fought a guerrilla war. The Vietcong hung on the belts
            of the Americans. The Americans struggled to identify the enemy because they
            didn’t wear uniforms and hid in the jungle and launched ambush attacks. The
            Americans could not be seen to be losing so they kept sending more forces and
            used new methods of attack such as Agent Orange to clear the forest.
            Level 2: Simple explanation of stated factor or other factor(s) 5–8
            Answer demonstrates specific knowledge and understanding that is
            relevant to the question
            Answer demonstrates a simple, sustained line of reasoning which is coherent,
            structured, substantiated and explicitly relevant.
            Answers arguing a preference for one judgement but with only basic
            explanation of another view will be marked at this level.
            Students may progress from a basic explanation of causation by simple
            reasoning and supporting it with factual knowledge and understanding.
            For example, Operation Rolling Thunder affected the conflict in Vietnam in the
            1960s. This was a bombing campaign by America that targeted government
            buildings in North Vietnam. The campaign was meant to last for a few weeks but
            it lasted for several years.
            Level 1: Basic explanation of one or more factors 1–4
            Answer demonstrates basic knowledge and understanding that is relevant
            to the question
            Answer demonstrates a basic line of reasoning, which is coherent, structured
            with some substantiation; the relevance might be implicit.
            Students recognise and provide a basic explanation of one or more factors.
            Students may offer a basic explanation of the stated factor, such as North
            Vietnam attacked a US warship in the Gulf of Tonkin.
            Students may offer basic explanations of other factor(s), for example, the US
            army could not defeat the guerrilla tactics of the Vietcong.
            Students either submit no evidence or fail to address the question 0
            """.strip()
        ],
        "option_e":
        [
            """
            Level 4: Complex explanation of stated factor and other factor(s) leading 13–16
            to a sustained judgement
            Answer demonstrates a range of accurate and detailed knowledge and
            understanding that is relevant to the question
            Answer demonstrates a complex, sustained line of reasoning which has a
            sharply-focused coherence and logical structure that is fully substantiated, with
            well-judged relevance.
            Extends Level 3.
            Students may progress from a developed explanation of causation by complex
            explanation of the relationship between causes supported by detailed factual
            knowledge and understanding to form a sustained judgement.
            For example, the Iran-Iraq war was a symptom rather than the cause of
            tensions. The war reflected the religious split between hostile Sunni and Shia
            states; both sides were led by regimes which had ambitions for status and
            power given the Iranian Revolution and Saddam’s arrogant and aggressive
            personality. Further instability was caused by Western interests, and the
            demand for oil. US behaviour fuelled suspicion across the Arab world,
            supporting Saddam and Israel at the same time. The war reflected long term
            tensions which explain its length and bitterness.
            Level 3: Developed explanation of the stated factor and other factor(s) 9–12
            Answer demonstrates a range of accurate knowledge and understanding
            that is relevant to the question
            Answer demonstrates a developed, sustained line of reasoning which has
            coherence and logical structure; it is well substantiated, and with sustained,
            explicit relevance.
            Extends Level 2.
            Answer may suggest that one reason has greater merit.
            Students may progress from a simple explanation of causation with developed
            reasoning supported by factual knowledge and understanding.
            For example, there was tension in the Gulf because, following the war, areas of
            dispute had not been settled such as shared use of the Shatt al-Arab waterway,
            which both countries needed for exports for oil by sea. And when Khomeini
            died in June 1989, his son vowed to continue the Islamic Revolution and hostility
            towards Iraq.
            For example, there were other main reasons for tension; there were underlying
            religious differences between Sunnis and Shias which were worsened by the
            interference of the West in the region. The West’s need for oil and continuing
            support for Israel against hostile Arab neighbours were bound to cause
            instability.
            Level 2: Simple explanation of stated factor or other factor(s) 5-8
            Answer demonstrates specific knowledge and understanding that is
            relevant to the question
            Answer demonstrates a simple, sustained line of reasoning which is coherent,
            structured, substantiated and explicitly relevant.
            Answers arguing a preference for one judgement but with only basic
            explanation of another view will be marked at this level.
            Students may progress from a basic explanation of causation by simple
            reasoning and supporting it with factual knowledge and understanding.
            For example, Saddam had survived, and he remained an important threat to
            Gulf stability because he was rearming to reassert his power and status in the
            region. Dealing with Iraqi debt might mean aggression against other oil rich
            states.
            Level 1: Basic explanation of one or more factors 1–4
            Answer demonstrates basic knowledge and understanding that is relevant
            to the question
            Answer demonstrates a basic line of reasoning, which is coherent, structured
            with some substantiation; the relevance might be implicit.
            Students recognise and provide a basic explanation of one or more factors.
            Students may offer a basic explanation of the stated factor, such as Iran and
            Iraq would remain enemies because of religious differences between Sunnis
            and Shias.
            Students may offer basic explanations of other factor(s), for example, western
            powers were concerned about oil supplies.
            Students either submit no evidence or fail to address the question 0
            """.strip()

        ]
    }
}

In [ ]:
print(aqa_hist_ms["judgement"]["option_e"][0])

In [ ]:
structure_ms_prompt = """
Format the Mark Scheme Text supplied below according to the following YAML structure:

Level_number:
  description: 
    - Description of Marking Level
  marks_awarded:
    maximum: int
    minimum: int
  criteria:
    - Make a list for each criteria that earns a mark
  progression: 
    - If relevant, how can student progress from previous level to current level
  example_answers: 
    -

Do not invent anything, extract and format the data only. If there is no information to fall into a YAML section, do not include that section.

Mark Scheme Text:
{mark_scheme_text}
"""

llm=ChatOpenAI(model_name="gpt-4o")

response = run_prompt_chain(
  prompt_template=structure_ms_prompt,
  llm=llm,
  mark_scheme_text=aqa_hist_ms["judgement"]["option_e"][0]
)

In [ ]:
print(response["prompt_chain_response"])

In [ ]:
# Cost of restructuring
print(response["prompt_chain_token_costing"]["total_cost"])

# Restructure Edexcel Business Studies Mark Scheme

In [ ]:
edexcel_business_studies_mark_scheme_texts = [
"""
Award 1 mark for identification of a way, plus 2 further marks for
explaining this way up to a total of 3 marks.
The business could choose a cheaper location (1). This is because
the business can use e-commerce (1). As a result, the business
may not require a physical retail location (1).
The business will need to be in a location with good internet
connections (1). This is because a business may need the internet
to communicate with stakeholders (1). A location with a weak
signal could cause barriers to communication for the business (1).
Accept any other appropriate response. Answers that list more
than one way with no explanation will be awarded a maximum of
1 mark.""",
"""
Award 1 mark for identification of a way, plus 2 further marks for
explaining this way up to a total of 3 marks.
A small business could gain an advantage by charging lower prices
(1). This would mean customers are more likely to buy the
product because it is cheaper than other products (1). Therefore,
this would attract customers from its competitors as they want to
save money (1).
A small business could improve the quality of the product (1). This
would attract customers because they think they are getting a
better product (1). As a result, the business would gain a better
reputation than competitors (1).
Accept any other appropriate response. Answers that list more
than one way with no explanation will be awarded a maximum of
1 mark.""",
"""
Award 1 mark for identification of an advantage, plus 2 further
marks for explaining this advantage up to a total of 3 marks.
A small business would not get sued for breaking the law (1). This
means the business would not have to face financial consequences
such as a fine (1). This will prevent the business from having a
negative image (1).
Employees will feel that the business is taking good care of them
(1). Therefore, the business will have a more motivated workforce
(1). This could lead to an increase in productivity (1).
Accept any other appropriate response. Answers that list more
than one advantage with no explanation will be awarded a
maximum of 1 mark.""",
"""
Award 1 mark for identification of a disadvantage, plus 2 further
marks for explaining this disadvantage, up to a total of 3 marks.
One disadvantage is that the employees will want to leave the
business (1). This means the business may have to find new
employees (1). This may increase the recruitment costs of the
business (1).
The employees may not be motivated (1). This means the
productivity of the employees will fall (1). Therefore, the business
may experience problems meeting customer orders (1).
Accept any other appropriate response. Answers that list more
than one disadvantage with no explanation will be awarded a
maximum of 1 mark.""",
"""
Award 1 mark for identification of an advantage, plus 2
further marks for explaining this advantage up to a total of 3
marks.
Retained profit is a cheap source of finance (1). This is because
interest is not paid on retained profit (1). Therefore, the business
would not experience an increase in costs (1).
Retained profit does not dilute the ownership of the business (1).
This is because the owner(s) keep full control of all decisions made
since no new shares are issued (1). This may lead to decisions
being made quicker in the business (1).
Accept any other appropriate response. Answers that list more
than one advantage with no explanation will be awarded a
maximum of 1 mark.
""",
"""
Question
number
Indicative content Mark
3(e)
 Costs are lower as there is less need for retail outlets (AO1b).
 Increased reach of the business - can sell beyond the
immediate locality (AO1b).
 This increases the potential profitability of the business as it
will not have to pay rent on shops and other premises (AO3a).
 This will lead to higher sales and therefore higher revenue.
Providing costs do not increase by more, profits will increase
(AO3a).
(6)
AO1b=3
AO3a=3
Level Mark Descriptor
0 No rewardable material.
Level 1 1–2 • Demonstrates elements of knowledge and understanding of
business concepts and issues, with limited business terminology
used (AO1b).
 Attempts to deconstruct business information and/or issues,
finding limited connections between points (AO3a).
Level 2 3–4 • Demonstrates mostly accurate knowledge and understanding of
business concepts and issues, including appropriate use of
business terminology in places (AO1b).
• Deconstructs business information and/or issues, finding
interconnected points with chains of reasoning, although there
may be some logical inconsistencies (AO3a).
Level 3 5–6 • Demonstrates accurate knowledge and understanding of
business concepts and issues throughout, including appropriate
use of business terminology (AO1b).
• Deconstructs business information and/or issues, finding
detailed interconnected points with logical chains of reasoning
(AO3a).
""",
"""
Question
number
Indicative content Mark
7(d)
 Buying its tea from cheaper suppliers will be a good option for
Good and Proper Tea because this will allow it to keep its
prices as low as possible in its competitive market (AO2).
 As a high quality, luxury product, Good and Proper Tea should
be able to charge higher prices for its tea without deterring its
market segments (AO2).
 This means Good and Proper Tea could target a wider range of
customers and different market segments, rather than being a
high priced, café and product (AO3a).
 The type of customer that buys expensive, gourmet tea, will
not be put off by an increase in price. They choose Good and
Proper Tea because of the quality and provenance of its
products (AO3a).
 However, Good and Proper Tea needs to be aware that the
market for coffee shops and cafes is very competitive and it
must avoid becoming one of a similar type of retailer.
Changing suppliers to reduce costs may damage its reputation
(AO3b).
 However, Good and Proper Tea does need to pay some
attention to the market and cannot continually increase the
price of its tea. This risks putting off customers who might
otherwise have tried its teas and potentially become repeat
customers (AO3b).
(9)
AO2=3
AO3a=3
AO3b=3
Level Mark Descriptor
0 No rewardable material.
Level 1 1–3 • Limited application of knowledge and understanding of business
concepts and issues to the business context (AO2).
• Attempts to deconstruct business information and/or issues,
finding limited connections between points (AO3a).
• Makes a judgement, providing a simple justification based on
limited evaluation of business information and issues relevant to
the choice made (AO3b).
Level 2 4-6 • Sound application of knowledge and understanding of business
concepts and issues to the business context although there may
be some inconsistencies (AO2).
• Deconstructs business information and/or issues, finding
interconnected points with chains of reasoning, although there
may be some logical inconsistencies (AO3a).
• Makes a judgement, providing a justification based on sound
evaluation of business information and issues relevant to the
choice made (AO3b).
Level 3 7-9 • Detailed application of knowledge and understanding of
business concepts and issues to the business context
throughout (AO2).
• Deconstructs business information and/or issues, finding
detailed interconnected points with logical chains of reasoning
(AO3a).
• Makes a judgement, providing a clear justification based on a
thorough evaluation of business information and issues relevant
to the choice made (AO3b).
""",
"""
Question
number
Indicative content Mark
6(c)
• Building a new factory in the UK will mean lower transport
costs (AO2).
• Expanding the Chinese factory is sensible as this factory
already has the employees with the skills and expertise in
manufacturing the business’ bikes (AO2).
• This means that overall costs can be kept down so that Frog
Bikes can charge prices which are competitive against rivals
such as Halfords and JE James (AO3a).
• This expertise means that bikes can be produced more cheaply
than in a UK factory, even when transportation costs from
China are taken into account (AO3a).
• The most appropriate option for Frog Bikes is to build the UK
factory. In addition to the costs benefit, the reputational
benefit it will gain will help to give a competitive advantage
over other retailers such as Halfords (AO3b).
• The best option is to stick with the Chinese factory. The
business is doing well and is becoming more successful, so
this factory, and its staff, are getting things right. Shifting to
the UK could be expensive as well as destabilising (AO3b).
(9)
AO2=3
AO3a=3
AO3b=3
Level Mark Descriptor
0 No rewardable material.
Level 1 1–3 • Limited application of knowledge and understanding of business
concepts and issues to the business context (AO2).
• Attempts to deconstruct business information and/or issues,
finding limited connections between points (AO3a).
• Makes a judgement, providing a simple justification based on
limited evaluation of business information and issues relevant to
the choice made (AO3b).
Level 2 4-6 • Sound application of knowledge and understanding of business
concepts and issues to the business context although there may
be some inconsistencies (AO2).
• Deconstructs business information and/or issues, finding
interconnected points with chains of reasoning, although there
may be some logical inconsistencies (AO3a).
• Makes a judgement, providing a justification based on sound
evaluation of business information and issues relevant to the
choice made (AO3b).
Level 3 7-9 • Detailed application of knowledge and understanding of
business concepts and issues to the business context
throughout (AO2).
• Deconstructs business information and/or issues, finding
detailed interconnected points with logical chains of reasoning
(AO3a).
• Makes a judgement, providing a clear justification based on a
thorough evaluation of business information and issues relevant
to the choice made (AO3b).
""",
"""
Award up to 2 marks for linked points outlining an impact of
changing consumer income on Good and Proper Tea. Award a
maximum of 1 mark if points are not linked.
If incomes are rising people will have more disposable income
(1). This will mean that luxuries, such as Good and Proper
Tea, are more likely to become popular and sales may
increase (1).
Falling incomes means people will have less disposable
income (1) and may resort to cheaper types of tea, rather
than higher quality products like Good and Proper Tea (1).
Do not accept impacts that would not be appropriate for a Good
and Proper Tea. For example, the demand for all foods will rise.
"""
]

In [ ]:
structure_edexcel_business_ms_prompt1 = """
Format the Mark Scheme Text supplied below according to the following YAML structure:

marking_criteria:
	- Make a list for each criteria that earns a mark
example_answers:
	-
marking_notes:
	-
total_marks:
	- 

Do not invent anything, extract and format the data only. If there is no information to fall into a YAML section, do not include that section.

Mark Scheme Text:
{mark_scheme_text}
""".strip()

In [ ]:
structure_edexcel_business_ms_prompt2 = """
Format the Mark Scheme Text supplied below according to the following YAML structure:
Indicative_content:
  -list all the indicative content given here
Level_number:
  criteria:
    - Make a list for each criteria at this level
  marks_awarded:
    maximum: int
    minimum: int
Do not invent anything, extract and format the data only. Include all text, do not leave anything out. Do not repea
 
Mark Scheme Text:
{mark_scheme_text}
""".strip()

In [ ]:
llm=ChatOpenAI(model_name="gpt-4o")

response = run_prompt_chain(
    prompt_template=structure_edexcel_business_ms_prompt1,
    llm=llm,
    mark_scheme_text=edexcel_business_studies_mark_scheme_texts[-1]
)

In [ ]:
print(response["prompt_chain_response"])

In [ ]:
print(response["prompt_chain_token_costing"]["total_cost"])